In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, get_peft_model, LoraConfig, TaskType  # 0.4.0
import torch
import argparse
from tqdm import tqdm 
from accelerate import Accelerator

accelerator = Accelerator()

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4"

In [5]:
# model_name = "google/gemma-2-27b-it"
model_name = "meta-llama/Llama-3.1-70B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name, trust_remote_code=True, 
    # quantization_config=bnb_config,
    device_map="auto"
)

model.model_parallel = True

tokenizer = AutoTokenizer.from_pretrained(model_name,     
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto"
)


tokenizer.padding_side = "left"
model = model.eval()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
import json
import subprocess
import re

def grep(f_name, kw):
    txt = ""
    with open(f_name, 'r') as f:
        for line in f.readlines():
            if kw in line:
                txt += line + "\n"
    return txt


def evaluate_qa_pair(qa_pair):
    doc_path = qa_pair["doc_path"]
    context_id = qa_pair["contextID"][0]  # Assuming contextID is always a list with one element
    query = qa_pair["query"]
    true_answer = qa_pair["raw_answer"]
    doc_path = "XBRLBench/XBRL_Filings/DowJones30/" + qa_pair["doc_path"]

    len_limit = 70000

    try:
        xml_lines = grep(doc_path, context_id)
    except:
        return False
        
    xml_lines = xml_lines[:len_limit]

    system_prompt = """You are a knowledgeable XBRL assistant that can answer questions based on XML data. 
                 You will be provided with a context extracted from an XBRL file and a question related to it. The example question can help you to learn the format of the answer.
                 Your task is to analyze the XBRL context and provide an accurate and very concise answer to the question, dont output xml or create new question.  """

    # Construct the user prompt with the specific context and question
    example_query = "What is the US GAAP XBRL tag for Cash and Cash Equivalents as reported by Example Company Inc for the Fiscal Year ending in FY 2022?"
    example_answer = "us-gaap:CashAndCashEquivalentsAtCarryingValue"

    query = re.sub(r'\([^()]*\)', '', query)
    # Construct the user prompt with the 1-shot example, context, and question
    user_prompt = f"""
    \nXBRL file:\n{xml_lines}
    \nExample question: {example_query}
    \nExample answer: {example_answer}
    \nQuestion: {query}
    \nAnswer:"""


    # 3. Generate answer from the LLM
    tokens = tokenizer(system_prompt + user_prompt, return_tensors="pt")
    tok_len = len(tokens.input_ids[0])
    print(len(tokens.input_ids[0]))
    for k in tokens.keys():
        tokens[k] = tokens[k].cuda()
    
    outputs = model.generate(**tokens, max_length=tok_len+30, pad_token_id=tokenizer.eos_token_id)
    
    llm_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    llm_answer = llm_answer.split("\nAnswer:")[1]
    llm_answer = llm_answer.replace("\n", "").strip()
    # 4. Compare the LLM's answer with the true answer
    print(f"LLM Answer : {llm_answer}")
    print(f"True Answer: {true_answer}")
    return llm_answer == true_answer or true_answer in llm_answer

model.generation_config.pad_token_id = tokenizer.pad_token_id
with open("qa_data.json", "r") as f:
    qa_data = json.load(f)
    qa_data = qa_data[:48]
    correct_count = 0
    total_count = len(qa_data)
    for qa_pair in tqdm(qa_data):
        is_correct = evaluate_qa_pair(qa_pair)
        if is_correct:
            correct_count += 1

    accuracy = (correct_count / total_count) * 100
    print(f"\nOverall Accuracy: {accuracy:.2f}%")


  0%|                                                                                                                                                                                                                                                   | 0/48 [00:00<?, ?it/s]

12918


  2%|████▉                                                                                                                                                                                                                                      | 1/48 [00:17<13:30, 17.25s/it]

LLM Answer : Note: The provided XBRL file is quite extensive and contains various financial data for Verizon Communications Inc. To answer the question, we need to locate
True Answer: us-gaap:AccountsPayableCurrent
15311


  4%|█████████▊                                                                                                                                                                                                                                 | 2/48 [00:38<14:49, 19.33s/it]

...

LLM Answer : The contextRef is c-70. The tag for Total Equity is stockholdersEquity. The correct US GAAP XBRL tag is:
True Answer: us-gaap:StockholdersEquityIncludingPortionAttributableToNoncontrollingInterest
19681


 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏    | 47/48 [18:13<00:22, 22.18s/it]

LLM Answer : us-gaap:OperatingIncomeLoss. Explanation: The tag is obtained by analyzing the XBRL file provided. The tag for Operating Income Loss
True Answer: us-gaap:OperatingIncomeLoss
16722


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [18:36<00:00, 23.25s/it]

LLM Answer : ```markdownus-gaap:AssetsCurrent``` The final answer is: us-gaap:AssetsCurrent. ```markdown
True Answer: us-gaap:AssetsCurrent

Overall Accuracy: 58.33%
